We start importing the pandas library, and set the number of maximally displayed rows to 5:

In [2]:
import pandas as pd
pd.options.display.max_rows = 5

## Data with delimiters

First, we will import some data on flowers from the provided data sets. Let's have a look at the file:

In [2]:
# for Linux: !head 02_text1.csv
!type 02_text1.csv

Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
5.1,3.5,1.4,0.2,setosa
4.9,3,1.4,0.2,setosa
4.7,3.2,1.3,0.2,setosa
4.6,3.1,1.5,0.2,setosa
5,3.6,1.4,0.2,setosa
5.4,3.9,1.7,0.4,setosa
4.6,3.4,1.4,0.3,setosa
5,3.4,1.5,0.2,setosa
4.4,2.9,1.4,0.2,setosa
4.9,3.1,1.5,0.1,setosa
5.4,3.7,1.5,0.2,setosa
4.8,3.4,1.6,0.2,setosa
4.8,3,1.4,0.1,setosa
4.3,3,1.1,0.1,setosa
5.8,4,1.2,0.2,setosa
5.7,4.4,1.5,0.4,setosa
5.4,3.9,1.3,0.4,setosa
5.1,3.5,1.4,0.3,setosa
5.7,3.8,1.7,0.3,setosa
5.1,3.8,1.5,0.3,setosa
5.4,3.4,1.7,0.2,setosa
5.1,3.7,1.5,0.4,setosa
4.6,3.6,1,0.2,setosa
5.1,3.3,1.7,0.5,setosa
4.8,3.4,1.9,0.2,setosa
5,3,1.6,0.2,setosa
5,3.4,1.6,0.4,setosa
5.2,3.5,1.5,0.2,setosa
5.2,3.4,1.4,0.2,setosa
4.7,3.2,1.6,0.2,setosa
4.8,3.1,1.6,0.2,setosa
5.4,3.4,1.5,0.4,setosa
5.2,4.1,1.5,0.1,setosa
5.5,4.2,1.4,0.2,setosa
4.9,3.1,1.5,0.2,setosa
5,3.2,1.2,0.2,setosa
5.5,3.5,1.3,0.2,setosa
4.9,3.6,1.4,0.1,setosa
4.4,3,1.3,0.2,setosa
5.1,3.4,1.5,0.2,setosa
5,3.5,1.3,0.3,setosa
4.5,2.3,1.3,0.3,setosa
4.

The first row contains the header (column names), and the columns are separated by a colon. The first 4 columns are floating point numbers, the last column is a string.

The pandas read method `read_csv()` tries to guess all these settings:

In [3]:
pd.read_csv('02_text1.csv')

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
...,...,...,...,...,...
148,6.2,3.4,5.4,2.3,virginica
149,5.9,3.0,5.1,1.8,virginica


If some guess goes wrong, the parameters can be specified explicitely:

In [4]:
pd.read_csv("02_text1.csv", sep = ",", header = 0,
            dtype = {'Sepal.Length' : float, 'Species' : str})

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
...,...,...,...,...,...
148,6.2,3.4,5.4,2.3,virginica
149,5.9,3.0,5.1,1.8,virginica


## Fixed-width data

Text columns can be a mess when not properly quoted:

In [5]:
# for Linux: !cat 02_text2.txt
!type 02_text2.txt

David Meyer  HÃ¶chstÃ¤dtplatz 6, 1200 Wien        0699 12345674  
Hugo H. Wolf An den langen LÃ¼ssen 47; 1190 Wien +43 4545 454545


In this case, we need to specify the column widths, or start/end of the columns.

In [6]:
data = pd.read_fwf('02_text2.txt', widths = [13, 35, 15], names = ['Name', 'Address', 'Telephone'], encoding = "UTF-8")
data

,Name,Address,Telephone
0,David Meyer,"Höchstädtplatz 6, 1200 Wien",0699 12345674
1,Hugo H. Wolf,An den langen Lüssen 47; 1190 Wien,+43 4545 454545


# Separating / Joining columns

In the example above, we need some further cleaning: the address should be separated into Street, ZIP code and City. We will try to split the street from the rest using the colon, and then separate ZIP and city with the space.

First, we replace the semi-colon with a colon:

In [7]:
data.Address = data.Address.str.replace(';', ',')
data

,Name,Address,Telephone
0,David Meyer,"Höchstädtplatz 6, 1200 Wien",0699 12345674
1,Hugo H. Wolf,"An den langen Lüssen 47, 1190 Wien",+43 4545 454545


Now, we extract the address strings as a list, and use the `partition()` method to separate the data:

In [8]:
parts = data.Address.str.partition(', ')
parts

,0,1,2
0,Höchstädtplatz 6,",",1200 Wien
1,An den langen Lüssen 47,",",1190 Wien


This results in a data frame with the three columns. We add the firt column (address) as a new `Street` column:

In [9]:
data['Street'] = parts[0]
data

,Name,Address,Telephone,Street
0,David Meyer,"Höchstädtplatz 6, 1200 Wien",0699 12345674,Höchstädtplatz 6
1,Hugo H. Wolf,"An den langen Lüssen 47, 1190 Wien",+43 4545 454545,An den langen Lüssen 47


Then, we further split the third column...

In [10]:
tmp = parts[2].str.partition(' ')
tmp

,0,1,2
0,1200,,Wien
1,1190,,Wien


... and add the result as `ZIP` and `City` columns. Finally, we delete the old `Address` column:

In [11]:
data['ZIP'] = tmp[0]
data['City'] = tmp[2]
data.drop('Address', axis = 1, inplace = True)
data

,Name,Telephone,Street,ZIP,City
0,David Meyer,0699 12345674,Höchstädtplatz 6,1200,Wien
1,Hugo H. Wolf,+43 4545 454545,An den langen Lüssen 47,1190,Wien


Recombining columns is easy, using the `+` operator:

In [12]:
data['Address'] = data.ZIP + " " + data.City + ", " + data.Street
data.drop(['ZIP', 'City', 'Street'], axis = 1)

,Name,Telephone,Address
0,David Meyer,0699 12345674,"1200 Wien, Höchstädtplatz 6"
1,Hugo H. Wolf,+43 4545 454545,"1190 Wien, An den langen Lüssen 47"


# Wide and Long format

Consider the following `USArrests` data about the arrests for certain crimes in various US states:

In [13]:
USArrests = pd.read_csv("02_USArrests.csv")
USArrests

,State,Murder,Assault,Rape
0,Alabama,13.2,236,21.2
1,Alaska,10.0,263,44.5
...,...,...,...,...
48,Wisconsin,2.6,53,10.8
49,Wyoming,6.8,161,15.6


The colums `Murder`, `Assault` and `Rape` are three different types of crime. A cleaner structure would be a string column `Crime`, along with a numeric column `Arrests`:

In [14]:
US_long = USArrests.melt(id_vars = ['State'], var_name = 'Crime', value_name='Arrests')
US_long

,State,Crime,Arrests
0,Alabama,Murder,13.2
1,Alaska,Murder,10.0
...,...,...,...
148,Wisconsin,Rape,10.8
149,Wyoming,Rape,15.6


This representation is called *long* format. To transform it back to the initial *wide* format, we use the `pivot()` method:

In [15]:
US_long.pivot(index = 'State', columns = 'Crime', values = 'Arrests')

Crime,Assault,Murder,Rape
State,,,
Alabama,236.0,13.2,21.2
Alaska,263.0,10.0,44.5
...,...,...,...
Wisconsin,53.0,2.6,10.8
Wyoming,161.0,6.8,15.6


# Missing data

Consider the following data on passengers of the Titanic disaster:

In [16]:
# for Linux: !cat 02_text3.txt
!type 02_text3.txt

Class Sex    Age    Died  Survived         
1st   Male   Child  0      5
             Adult  118    57
      Female Child  0      1 
             Adult  4      140
2nd   Male   Child  0      11
             Adult  154    14
      Female Child  0      13
             Adult  13     80
3rd   Male   Child  35     13
             Adult  387    75
      Female Child  17     14
             Adult  89     76
Crew  Male   Child  0      ??
             Adult  670    192
      Female Child  0      ??
             Adult  3      20


The Class and Sex labels have not been repeated in each row, and there are two missing values (`??`) for Children in the Crew Class. When we read in the data using `read_fwf()`, we get:

In [17]:
data = pd.read_fwf('02_text3.txt', na_values='??')
data

,Class,Sex,Age,Died,Survived
0,1st,Male,Child,0,5.0
1,NaN,NaN,Adult,118,57.0
...,...,...,...,...,...
14,NaN,Female,Child,0,NaN
15,NaN,NaN,Adult,3,20.0



Both empty cells and missing values are represented by the `NaN` symbol. To clean the data, we need to replace `NaN` in the `Class` and `Sex` columns with the last known label from the top, and to handle the "real" missing values suitably. We start handling the missings in the `Class` and `Sex` columns using the `fillna()` method:

In [18]:
data[['Class', 'Sex']] = data[['Class', 'Sex']].fillna(method = "ffill")
data

,Class,Sex,Age,Died,Survived
0,1st,Male,Child,0,5.0
1,1st,Male,Adult,118,57.0
...,...,...,...,...,...
14,Crew,Female,Child,0,NaN
15,Crew,Female,Adult,3,20.0


The remaining `NaN` values could either be removed, or replaced by some sensible value. If the values were missing "completely at random", we could, e.g., use the mean of the other values. In this case, however, these values are not missing at random: Crew members clearly were all adult, so the correct replacement value is 0.

To filter all rows with missing values, we can use the `isna()` method, indicating, for each cell, whether the value is missing or not. Applying `any()` to the *transposed* table will give us, for each row, whether at least one value is missing:

In [19]:
data.isna().T.any()

0     False
1     False
      ...  
14     True
15    False
Length: 16, dtype: bool

Thus, all incomplete rows are given by:

In [20]:
data[data.isna().T.any()]

,Class,Sex,Age,Died,Survived
12,Crew,Male,Child,0,NaN
14,Crew,Female,Child,0,NaN


These could be removed using `dropna()`:

In [21]:
data.dropna()

,Class,Sex,Age,Died,Survived
0,1st,Male,Child,0,5.0
1,1st,Male,Adult,118,57.0
...,...,...,...,...,...
13,Crew,Male,Adult,670,192.0
15,Crew,Female,Adult,3,20.0


... or replaced with 0 again using `fillna()`:

In [22]:
data.fillna(0, inplace = True)
data

,Class,Sex,Age,Died,Survived
0,1st,Male,Child,0,5.0
1,1st,Male,Adult,118,57.0
...,...,...,...,...,...
14,Crew,Female,Child,0,0.0
15,Crew,Female,Adult,3,20.0


__Final note:__ because `NaN` is only available for floating point values, the data type of `Survived` is incorrect. To fix this, we can use:

In [23]:
data.Survived = data.Survived.astype(int)
data

,Class,Sex,Age,Died,Survived
0,1st,Male,Child,0,5
1,1st,Male,Adult,118,57
...,...,...,...,...,...
14,Crew,Female,Child,0,0
15,Crew,Female,Adult,3,20
